In [2]:
#coding:utf-8
import networkx as nx
from networkx.algorithms import bipartite
import pandas as pd
import numpy as np
import re
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import os
import cn2an
import chinese2digits as c2d
import operator
import plotly.express as px
import plotly
import plotly.graph_objects as go
import plotly as py
import plotly.offline as offline
offline.init_notebook_mode(connected=True)
from urllib.request import urlopen
import json
import requests
from plotly.subplots import make_subplots
from collections import Counter
from ipynb.fs.full.case_to_graph_only_method import convert_csv_graph, provinces_map

In [3]:
base_url = "/Users/starice/Desktop/total_extracted_result/"
pre_dir = ['type1', 'type2', 'type3', 'type4']
dir_name = ['2014', '2015', '2016', '2017', '2018', '2019', '2020']
dir_sname = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
# csv_graph, _new_csvpd = convert_csv_graph(pre_dir[:], dir_name[:], dir_sname[:])
csv_graph = nx.read_gpickle("/Users/starice/Desktop/csv_graph.gpickle")

In [4]:
# 在后面绘制地区区域划分图会用到
# with open("/Users/starice/OwnFiles/cityu/RA/code/case_process/china_province.geojson") as f:
#     provinces_map = json.load(f)
mapbox_access_token = "pk.eyJ1Ijoic3RhcmljZSIsImEiOiJjazN6Y2s5dTUxY2R6M2xxcHllbXk4YWFzIn0.lActFqLzqRWGn7dqr4BShw"
px.set_mapbox_access_token(mapbox_access_token)

In [5]:
plaintiff_titles = ['原告', '上诉人(原审原告)', '上诉人(原审被告)', '上诉人(一审原告)', '上诉人(一审被告)', '上诉人', \
 '上诉人(原审被告、反诉原告)', '上诉人(原审第一被告)', '再审申请人(一审原告、二审上诉人)', \
 '再审申请人(一审原告、二审上诉人)', '申请再审人(原审被告)', '再审申请人(原审被告)', '申诉人(一审原告、二审上诉人)', \
 '申诉人(一审原告、二审上诉人、原再审申请人)', '再审申请人(原审原告)', '上上诉人(原审被告)', \
 '申诉人(一审原告、二审上诉人、再审申请人)', '再审申请人', '再审申请人(一审被告、二审上诉人)', '原告(反诉被告)', \
 '申诉人(原审原告)', '公益诉讼起诉人', '诉讼代表人']

defendant_titles = ['被上诉人一(原审被告一)', '被上诉人二(原审被告二)', '被申诉人(一审被告、二审被上诉人、再审被申请人)', \
 '被申请人(一审原告、二审被上诉人)', '被申诉人(原审被告)', '被申请人(一审被告)', '被申请人', \
 '被申诉人(一审被告、二审上诉人)', '被告(反诉原告)', '原审第二被告', '原审第三被告', '被申请人(一审被告、二审被上诉人)', \
 '第三被告', '一审被告', '被申请人(原审原告)', '被申请人(一审被告、二审上诉人)', '被申请人(原审被告)', '被告', \
 '被上诉人(原审被告)', '被上诉人(原审原告)', '原审被告', '被上诉人(一审被告)', '第一被告', '第二被告', '被告一', \
 '被告二', '被上诉人(一审原告)', '被上诉人(原审第三人)', '被上诉人', '被上诉人(原审原告、反诉被告)']

cases = [(n, d) for n, d in csv_graph.nodes(data=True) \
               if d['bipartite']==0]
plaintiffs = [(e1, e2, d) for e1, e2, d in csv_graph.edges(nbunch=[n[0] for n in cases], data=True) \
                    if d['title'] in plaintiff_titles]
defendants = [(e1, e2, d) for e1, e2, d in csv_graph.edges(nbunch=[n[0] for n in cases], data=True) \
                    if d['title'] in defendant_titles]

In [6]:
pd_plaintiffs = pd.DataFrame({"case_id": [i[0] for i in plaintiffs], "plaintiff": [i[1] for i in plaintiffs]})
pd_defendants = pd.DataFrame(({"case_id": [i[0] for i in defendants], "defendant": [i[1] for i in defendants]}))
pd_cases = pd.DataFrame({"case_id": [i[0] for i in cases], \
                        "judgement_date": [i[1]['judgement_date'] for i in cases], \
                         "is_success": [i[1]['is_success'] for i in cases], \
                         "lat": [i[1]['lat'] for i in cases], \
                         "lon": [i[1]['lon'] for i in cases], \
                         "court_name": [i[1]['court_name'] for i in cases], \
                         "procedure": [i[1]['procedure'] for i in cases], \
                         "judge": [i[1]['judge'] for i in cases], \
                         "legalfee": [i[1]['legalfee'] for i in cases], \
                         "objectmoney": [i[1]['objectmoney'] for i in cases], \
                         "province": [i[1]['province'] for i in cases], \
                         "city": [i[1]['city'] for i in cases], \
                         "reason": [i[1]['reason'] for i in cases], \
                         "district": [i[1]['district'] for i in cases], \
                         "penalty": [i[1]['penalty'] for i in cases]})
pd_cases['year'] = pd.DatetimeIndex(pd_cases['judgement_date']).year
pd_cases['month'] = pd.DatetimeIndex(pd_cases['judgement_date']).month
pd_cases['day'] = pd.DatetimeIndex(pd_cases['judgement_date']).day
all_cases = pd_cases.merge(pd_plaintiffs, on="case_id", how="left")
all_cases = all_cases.merge(pd_defendants, on="case_id", how="left")
all_cases.drop(all_cases[all_cases['plaintiff'].isin(["XX", "xx", "**"])].index, inplace=True) #delete the cases with invalid plaintiff name

In [7]:
#获取所有一审案件
first_cases = all_cases[all_cases['procedure']=="一审"]
print("一审案件数量： ", len(first_cases['case_id'].drop_duplicates()))

# 获取所有案件原告的节点度并排序
degree_1stdfs = first_cases.groupby("defendant")['case_id'].unique().reset_index()
degree_1stdfs['case_count'] = degree_1stdfs['case_id'].apply(lambda r: len(r))
degree_1stdfs.sort_values(by="case_count", inplace=True, ascending=False)
fds_200 = degree_1stdfs[:200]
fds_200

一审案件数量：  32528


,defendant,case_id,case_count
4099,天津市人人乐商业有限公司,"[57aa73eec2265c28a555d425, 581161a82f12c63192b...",2359
12245,重庆永辉超市有限公司,"[581136be2f12c63192b38df6, 57a951bfc2265c28a54...",1086
12048,重庆华润万家生活超市有限公司,"[57ad2054c2265c04d1192e9a, 581212002f12c63192c...",472
8669,浙江淘宝网络有限公司,"[57ab9058c2265c28a560195d, 57ab9686c2265c28a56...",374
6377,成都蓉李记餐饮管理有限公司,"[5df563a6ab936e4614986c02, 5ee076333edbe71b2f8...",370
...,...,...,...
9601,深圳沃尔玛百货零售有限公司中山分店,"[5a58c65de1382313cb9cc579, 5a58c675e1382313cb9...",24
11706,辽宁乐天超市有限公司,"[581218102f12c63192cb6549, 5834b3162f12c6784ea...",24
4177,天津永辉超市有限公司,"[5b13e899e138231bb66ced46, 5b13e89de138231bb66...",23
8067,沃尔玛华东百货有限公司,"[57aafd6dc2265c25897d2446, 58105d8b2f12c631928...",23


In [8]:
fds_10 = degree_1stdfs[:10]
new_selected_1stcp = first_cases[first_cases['defendant'].isin(fds_10['defendant'])]

In [13]:
degree_1stdfs[:30]
# 看一下被告到至少100次的被告是否有案件数量逐年下降的趋势

,defendant,case_id,case_count
4099,天津市人人乐商业有限公司,"[57aa73eec2265c28a555d425, 581161a82f12c63192b...",2359
12245,重庆永辉超市有限公司,"[581136be2f12c63192b38df6, 57a951bfc2265c28a54...",1086
12048,重庆华润万家生活超市有限公司,"[57ad2054c2265c04d1192e9a, 581212002f12c63192c...",472
8669,浙江淘宝网络有限公司,"[57ab9058c2265c28a560195d, 57ab9686c2265c28a56...",374
6377,成都蓉李记餐饮管理有限公司,"[5df563a6ab936e4614986c02, 5ee076333edbe71b2f8...",370
4139,天津市河东区宽巷子小吃店,"[5df563a6ab936e4614986c02, 5ee076333edbe71b2f8...",370
8150,沈阳家乐福商业有限公司,"[57aab194c2265c04d1012292, 581069ab2f12c631928...",319
3761,四川徽记食品股份有限公司,"[5a610070e1382319e2be679c, 5a6e72c3e13823c1bbc...",307
4111,天津市人人乐商业有限公司滨海新区河北路购物广场,"[5a303aede138234a89714d1f, 5a278215e1382327972...",290
12244,重庆永辉云创科技有限公司,"[5cf14cfa2dd6ef6bf0a3ddfd, 5cf14ced2dd6ef6bf0a...",283


In [10]:
time_nselected_1stcp = new_selected_1stcp.groupby(['defendant', 'year'])['case_id'].nunique().reset_index()
time_nselected_1stcp.rename(columns={"case_id": "case_count"}, inplace=True)

In [11]:
fig = px.box(
    time_nselected_1stcp, 
    x = "year", 
    y = "case_count", 
#     color = "defendant"
)
fig.update_yaxes(type="log")
fig.show()

In [12]:
fig = px.line(
    time_nselected_1stcp, 
    x = "year", 
    y = "case_count", 
    color = "defendant"
)
# fig.update_yaxes(type="log")
fig.show()